# Atomap tutorial: finding and analysing sublattices in perovskite oxide thin films

For more details see the open access article: **Atomap: a new software tool for the automated analysis of atomic resolution images using two-dimensional Gaussian fitting**. https://dx.doi.org/10.1186/s40679-017-0042-5

This tutorial shows how to use Atomap to analyse a La$_{0.7}$Sr$_{0.3}$MnO$_3$ (LSMO) + LaFeO$_3$ (LFO) bilayer film structure grown on SrTiO$_3$-111 (STO) substrate, where the aim is to visualize changes in the lattice across the films. For more information about the material, see https://doi.org/10.1103/PhysRevB.94.201115

For documentation about the Atomap software itself, see https://atomap.org/

## Importing the libraries

Firstly, we must set the plotting toolkit:

In [ ]:
%matplotlib qt5

Atomap relies heavily on HyperSpy for visualization and fitting, and uses HyperSpy signals for most of the outputs. So we need to import both HyperSpy and Atomap.

In [ ]:
import hyperspy.api as hs

In [ ]:
import atomap.api as am

### Loading data

Atomap uses HyperSpy signals as its input, which can be any loaded from many different types of files. DM3/DM4, tif, emi/ser, jpg or HDF5-files. This can be loaded using `s = hs.load(your_filename)`, here we'll be looking at the file `ADF_image.hspy`. Note that since this is a text string, we must use quotation marks.

In [ ]:
s = hs.load("ADF_image.hspy")

By using `plot()`, we can have a look at the bilayer and substrate system. There are two major types of atomic columns visible here across all the three layers: the brightest columns which are the A-cations (La/Sr, La, or Sr), and the second brightest columns (Mn, Fe or Ti).

In [ ]:
s.plot()

Based on the intensity in the A-cations, we can see there are three different structures:

- At the top is the LSMO film
- In the middle the LFO film, where we can also see a slight "ellipticity" of the A-cations
- At the bottom is the STO substrate

In addition to the A- and B-cations, we also have the oxygen columns, which aren't really visible here. To see them, we have a look at the Annular Bright Field (ABF) data, which was acquired simultaneously with the ADF image. 

<img src="srtio3_adf_abf.jpg" width=300 height=300 />

Load the `ABF_image.hspy`

In [ ]:
s_abf = hs.load("ABF_image.hspy")

Use `plot()` to visualize this dataset

In [ ]:
s_abf.plot()

Here we see that the contrast is inverted compared to the previous signal, which is expected for ABF.

The darkest atomic columns are the A-cations, the second darkest the B-cations, and the least dark the oxygen atomic columns.

Comparing the three layers, there is a clear difference in the middle, LFO, layer. Here we can see some kind of shift in the oxygen columns.

---------

Lets also have a look at a live FFT of the ADF dataset, utilizing HyperSpy's region of interest functionality.

In [ ]:
def get_live_fft(s):
    roi = hs.roi.RectangularROI()
    s.plot()
    s_roi = roi.interactive(s)
    s_fft = hs.interactive(s_roi.fft, shift=True)
    s_fft.plot(power_spectrum=True)
    return s_fft

Then use the ADF signal, `s` with `get_live_fft` function.

In [ ]:
get_live_fft(s)

Make the box small enough to cover just the middle, LFO, layer and move it up and down to see what changes.

In the powerspectrum of the FFT, there seems to be an extra "FFT spot" in the LFO layer.

----

So there is clearly some interesting things happening with the atomic structure in this bilayer film system, and we would like to quantify it for every atomic column!

Specifically:

1. Ellipticity of A-cations
2. Oxygen column shift

## Finding initial positions

Our first task is finding and fitting the most intense atomic columns. This is done by using a peak finder, where the minimum distance between the features is used as an input parameter. This function returns a HyperSpy signal, where the atom positions are saved as permanent markers in the metadata.

In [ ]:
s_peaks = am.get_feature_separation(s, pca=True)

In this plot, the minimum feature separation is shown on the x-axis for the navigation plot. This parameter can be changed by using the arrow keys. For this image, we want to get the peaks for only the most intense atomic columns.
A feature separation of 9 works fine.

In [ ]:
s_peaks.plot()

We use this value as an input for the next step, which involves getting these atomic positions as a list via the `get_atom_positions` function.

In [ ]:
atom_positions = am.get_atom_positions(s, separation=9, pca=True)

For other, more advanced structures, there are interactive functions which can be used to to add or remove atoms, use `am.add_atoms_with_gui`, see https://atomap.org/gui_functions.html#adding-atoms-using-gui for more info.

Also, atoms from different regions can be selected interactively using `am.select_atoms_with_gui`, see https://atomap.org/gui_functions.html#selecting-atoms-with-gui

## Making a sublattice

The `atom_positions` are used to initialize the sublattice object, together with the ADF signal `s`. This class object is found in `am.Sublattice`

Note: remember that almost all Python objects have docstrings, which is seen by putting a question mark after the function. For example: `s.plot?`

In [ ]:
sublattice_A = am.Sublattice(atom_position_list=atom_positions, image=s)

This sublattice object contains the atom positions we just found, the image, and many utility functions. For example a `plot` function.

In [ ]:
sublattice_A.plot()

The first step is to refine the atomic positions using center of mass. Since the initial atom positions are not very good.

Run `find_nearest_neighbors`, then `refine_atom_positions_using_center_of_mass`.

In [ ]:
sublattice_A.find_nearest_neighbors()

In [ ]:
sublattice_A.refine_atom_positions_using_center_of_mass()

Then refine with 2D Gaussians, using `refine_atom_positions_using_2d_gaussian`

In [ ]:
sublattice_A.refine_atom_positions_using_2d_gaussian()

This will give much more accurate atomic positions, visualize this using `plot`

In [ ]:
sublattice_A.plot()

## Constructing zone axes

The next step is to find the relation between the atom positions, by ordering them into atomic planes via `construct_zone_axes`

In [ ]:
sublattice_A.construct_zone_axes()

These atomic planes can be visualized via the `plot_planes`. Navigate using the arrow keys. The number to the top left is the direction of the zone axis.

In [ ]:
sublattice_A.plot_planes()

### A-cations analysis

#### Distance between atomic planes

This is visualized via the `get_monolayer_distance_map`, then plotting this signal.

In [ ]:
monolayer_a = sublattice_A.get_monolayer_distance_map()

In [ ]:
monolayer_a.plot()

Note that this visualization is based on interpolating the values into an image, which can create some difficult to interpret images when there is a variation between the atomic planes. So what we're seeing is consistent with the features we saw in the FFT.

A better way of visualizing this is by getting an average in the growth direction.

#### Distance between atomic planes in the growth direction

To do this, we need to know

1. Which atomic planes do we want
2. Which atomic plane do we want to be the "zero point"

The atomic plane "vector" for the 111-growth direction is the **third** one (which we can confirm using `sublattice_A.plot_planes()`). We then get this from `zone_axis_average_distances[2]`. Note the `2`, as Python is zero-indexed.

In [ ]:
zone_axis_111 = sublattice_A.zones_axis_average_distances[2]

Then, we grab the "zero point" which we set as the interface plane: `34` (see `plot_planes()`). We get this atomic plane via `atom_planes_by_zone_vector`, where the `zone_vector` is the one we just found.

In [ ]:
interface_layer_111 = sublattice_A.atom_planes_by_zone_vector[zone_axis_111][34]

In [ ]:
interface_layer_111

Next, `get_monolayer_distance_line_profile` is used with these two, to get the distance between the atomic planes in the growth distance.

In [ ]:
s_A_monolayer = sublattice_A.get_monolayer_distance_line_profile(zone_vector=zone_axis_111, atom_plane=interface_layer_111)

This returns a signal, which we visualize using `plot`.

In [ ]:
s_A_monolayer.plot()

To get the raw data for this, see `metadata.line_profile_data`

In [ ]:
s_A_monolayer.metadata.line_profile_data

#### Ellipticity

Seeing the ellipticity is easier, just use `get_ellipticity_map`.

In [ ]:
s_A_elli = sublattice_A.get_ellipticity_map()

Then plot it

In [ ]:
s_A_elli.plot()

To get a line profile, use `get_ellipticity_line_profile` with the `interface_layer_111` we found earlier.

In [ ]:
s_A_elli_line = sublattice_A.get_ellipticity_line_profile(interface_layer_111)

And plot it

In [ ]:
s_A_elli_line.plot()

To plot it as vectors, use `plot_ellipticity_vectors`

In [ ]:
sublattice_A.plot_ellipticity_vectors()

### Next step

Having analysed the A-cation "sublattice", with looking at the distances between the atomic columns and their ellipticity, we can now try to quantify the shifts in the oxygen columns.

However, this is a bit more tricky, as the oxygen columns are the least intense atomic column in the ABF data. Thus, we must first take into account the intensity from the A- and B-cations.

## Finding the oxygen

To do this, we use the A-cations we already found in the ADF image as initial positions in the ABF image.

However, the first step is to "invert" the contrast in the ABF image.

In [ ]:
data_abf_invert = 1 / s_abf.data

Then we grab the atom_positions in `sublattice_A`, and use it to make a new sublattice combined with the inverted ABF data.

In [ ]:
initial_position_A = sublattice_A.atom_positions

In [ ]:
sublattice_A2 = am.Sublattice(initial_position_A, image=data_abf_invert, color='r', name="A")

Then we run the same refinement as we did with the first sublattice.

In [ ]:
sublattice_A2.find_nearest_neighbors()
sublattice_A2.refine_atom_positions_using_center_of_mass()
sublattice_A2.refine_atom_positions_using_2d_gaussian()
sublattice_A2.construct_zone_axes()

Plotting it, to see if the fitting worked out

In [ ]:
sublattice_A2.plot()

Then, we "subtract" the intensity from the A-cations, to enable us to fit the less intense B-cations.

In [ ]:
from atomap.tools import remove_atoms_from_image_using_2d_gaussian

In [ ]:
image_without_A2 = remove_atoms_from_image_using_2d_gaussian(sublattice_A2.image, sublattice_A2)

#### Finding the B-cations

We utilize the fact that the B-cations are always between the A-cations in the 001 atomic planes. We see this in the `plot_planes()`.

In [ ]:
sublattice_A2.plot_planes()

This is the second `zones_axis_average_distances` index (`1`)

In [ ]:
zone_axis_001 = sublattice_A2.zones_axis_average_distances[1]
zone_axis_001

Then we pass this to the `find_missing_atoms_from_zone_vector`, which returns a new list of positions

In [ ]:
initial_positions_B = sublattice_A2.find_missing_atoms_from_zone_vector(zone_axis_001)

Which is used to make a sublattice for the B-cations

In [ ]:
sublattice_B2 = am.Sublattice(initial_positions_B, image=image_without_A2, color='b', name="B")

In [ ]:
sublattice_B2.plot()

In [ ]:
sublattice_B2.find_nearest_neighbors()
sublattice_B2.refine_atom_positions_using_center_of_mass()
sublattice_B2.refine_atom_positions_using_2d_gaussian()
sublattice_B2.construct_zone_axes()

Lets plot the result of this, to see if the fitting worked nicely.

In [ ]:
sublattice_B2.plot()

#### Finding the oxygen atomic column

Here, we utilize the fact that the oxygen columns are between the B-cation atomic columns in the 011 planes.

In [ ]:
sublattice_B2.plot_planes()

This is the 0th index in `zones_axis_average_distances`

In [ ]:
zone_axis_002 = sublattice_B2.zones_axis_average_distances[0]
O_positions = sublattice_B2.find_missing_atoms_from_zone_vector(zone_axis_002)
image_without_AB = remove_atoms_from_image_using_2d_gaussian(sublattice_B2.image, sublattice_B2)
sublattice_O = am.Sublattice(O_positions, image_without_AB, color='g', name="O")

Next, we plot this last sublattice, to see if everything worked out.

In [ ]:
sublattice_O.plot()

Lastly, we run the center of mass and 2D-Gaussian refinement.

In [ ]:
sublattice_O.construct_zone_axes()
sublattice_O.refine_atom_positions_using_center_of_mass()
sublattice_O.refine_atom_positions_using_2d_gaussian()

In [ ]:
sublattice_O.plot()

## The atom lattice object

Having made these three sublattices, we can combine them in an `Atom_Lattice` object

In [ ]:
atom_lattice = am.Atom_Lattice(image=s_abf.data, name='ABO3', sublattice_list=[sublattice_A2, sublattice_B2, sublattice_O])

Which also has a plotting function.

In [ ]:
atom_lattice.plot()

Atom_Lattice objects can be stored, to avoid having to run the often slow fitting routines several times.

In [ ]:
atom_lattice.save(overwrite=True)

Which can be restored:

In [ ]:
atom_lattice2 = am.load_atom_lattice_from_hdf5("ABO3_atom_lattice.hdf5")

#### Visualizing the oxygen shifts

Now we can finally visualize the oxygen shifts.

We grab the sublattices which is contained in the loaded `atom_lattice2`, which is in the `sublattice_list`.

First the A-sublattice:

In [ ]:
sublattice_A = atom_lattice2.sublattice_list[0]

Then the O-sublattice:

In [ ]:
sublattice_O = atom_lattice2.sublattice_list[2]

We then use these with the `get_atom_distance_difference_map` function in `sublattice_O` to visualize the oxygen shifts. `add_zero_value_sublattice` in used to avoid the interpolation issues seen earlier when visualizing this type of data.

In [ ]:
s_o_atom_difference = sublattice_O.get_atom_distance_difference_map(add_zero_value_sublattice=sublattice_A)

Here, the deviation from centrosymmetry is shown, giving a quantification of the oxygen shifts. This can then be used to estimate the tilt of the oxygen octahedrons.

<img src="atom_distance_difference.jpg" width=300 height=300 />

In [ ]:
s_o_atom_difference.plot()

## Determining polarization

In many ferroelectric materials, the spontaneous electric polarization can be determined by looking at the shift of some atomic columns in relation to the others. One example of this is in the ferroelectric perovskite oxides, where the B-cation is shifted from its cubic centrosymmetric position. The polarization can be determined by finding both the direction and magnitude of this shift.

Firstly, we get an appropriate artificial dataset, resembling a ferroelectric thin film grown on top of a non-ferroelectric substrate.

In [ ]:
atom_lattice = am.dummy_data.get_polarization_film_atom_lattice()
atom_lattice.plot()

The blue, B-cation, atom columns in the top part of the image are shifted towards the left. By finding the centre position of four neighboring red A-cation forming a square, this shift can be quantified.

Finding these neighbors relies on moving along two zone axis directions in the A-cation sublattice.

In [ ]:
sublatticeA = atom_lattice.sublattice_list[0]
sublatticeA.construct_zone_axes()

Next, find the two perpendicular zone axes spanning this square. For the perovskite oxide 100 projection, this is most likely the two first ones.

In [ ]:
sublatticeA.plot_planes()

The zone axes are then used with the B-cation sublattice:

In [ ]:
za0 = sublatticeA.zones_axis_average_distances[0]
za1 = sublatticeA.zones_axis_average_distances[1]
sublatticeB = atom_lattice.sublattice_list[1]
s_polarization = sublatticeA.get_polarization_from_second_sublattice(za0, za1, sublatticeB)

This can be visualized directly by using the ``plot`` method, and the data itself can be accessed in the signal’s metadata.

In [ ]:
s_polarization.plot()
vector_list = s_polarization.metadata.vector_list